In [1]:
import numpy as np
import pandas as pd
import pickle
import torch
import random

In [2]:
# data_filepath = '/n/dtak/mimic-pipeline-tools/intervention-onset/model-data/9-18-interpretable-timeseries.p'
data_filepath = '/n/home07/carissawu/intervention-onset/model-data/11-15-21-interpretable-timeseries-vasopressor.p'

In [3]:
# load from pickle

intervention_data = pickle.load( open(data_filepath, "rb" ) )

In [4]:
intervention_data[200011].columns

# Has_Interv is y
# Vaso_Time has the time of the intervention (if there is a time)

Index(['Times', 'Has_Interv', 'Interv_Time', 'Age', 'is_F', 'is_urgent',
       'is_emergency', 'is_sicu', 'is_tsicu', 'is_micu', 'is_csru', 'dbp',
       'dbp_ind', 'fio2', 'fio2_ind', 'GCS_eye', 'GCS_eye_ind', 'GCS_motor',
       'GCS_motor_ind', 'GCS_verbal', 'GCS_verbal_ind', 'hr', 'hr_ind', 'map',
       'map_ind', 'sbp', 'sbp_ind', 'spontaneousrr', 'spontaneousrr_ind',
       'spo2', 'spo2_ind', 'temp', 'temp_ind', 'urine', 'urine_ind', 'weight',
       'Weight', 'bun', 'bun_ind', 'magnesium', 'magnesium_ind', 'platelets',
       'platelets_ind', 'sodium', 'sodium_ind', 'alt', 'alt_ind', 'hct',
       'hct_ind', 'po2', 'po2_ind', 'ast', 'ast_ind', 'potassium',
       'potassium_ind', 'wbc', 'wbc_ind', 'bicarbonate', 'bicarbonate_ind',
       'creatinine', 'creatinine_ind', 'lactate', 'lactate_ind', 'pco2',
       'pco2_ind', 'bilirubin_total', 'bilirubin_total_ind', 'glucose',
       'glucose_ind', 'inr', 'inr_ind', 'hgb', 'hgb_ind', 'GCS', 'GCS_ind'],
      dtype='object')

In [55]:
# keep all of the columns in X_ind.
# standardize everything that is not an indicator.

In [6]:
intervention_data[200011]['Age']

0     84.231778
1     84.231778
2     84.231778
3     84.231778
4     84.231778
5     84.231778
6     84.231778
7     84.231778
8     84.231778
9     84.231778
10    84.231778
11    84.231778
12    84.231778
13    84.231778
14    84.231778
15    84.231778
16    84.231778
Name: Age, dtype: float64

In [17]:
ind_strs = ['_8ind', '_everind', '_ind']

In [18]:
ind_cols = []
for col in intervention_data[200011].columns:
    for ind_str in ind_strs:
        if ind_str in col:
            ind_cols.append(col)

In [19]:
to_drop = ['Times', 'Has_Interv', 'Interv_Time']
y_col = 'Has_Interv'

In [20]:
X_array = []
mask_array = []
y_array = []

for patient in intervention_data.keys():
    intervention_df = intervention_data[patient]    

    x_arr = intervention_df.drop(to_drop, axis=1)
    
    # mask will have only the indicators
    mask = intervention_df[ind_cols]
    
    y = np.array(intervention_df[y_col].astype(int))
    
    X_array.append(x_arr)
    mask_array.append(mask)
    y_array.append(y)

In [10]:
len(X_array)

15552

In [26]:
num_cols = len(X_array[0].columns)
cols = X_array[0].columns

In [27]:
len(cols)
X_array[0]['fio2_ind']

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     1.0
7     0.0
8     0.0
9     0.0
10    0.0
11    1.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
25    0.0
26    0.0
27    0.0
28    0.0
29    0.0
30    0.0
31    0.0
32    0.0
33    0.0
34    0.0
35    0.0
36    0.0
37    0.0
38    0.0
39    0.0
40    0.0
41    0.0
42    0.0
43    0.0
44    0.0
45    0.0
46    0.0
47    0.0
48    0.0
49    0.0
50    0.0
51    0.0
52    0.0
53    0.0
54    0.0
55    0.0
Name: fio2_ind, dtype: float64

In [13]:
num_cols

72

In [38]:
for c in cols:
    if c not in X_array[0].columns:
        print(c)

In [37]:
# first calculate the mean
sum_array = np.zeros(num_cols)
n = 0

for patient in X_array:
    sum_array += patient.sum(axis=0).values
    n += patient.shape[0]

In [40]:
# Store the weights of the first 100 patients.

weights = []

n = 0

for patient in X_array:
    weights.append(patient.iloc[0]['Weight'])
    
    n += 1
    
    if n % 10000 == 0:
        break
    

In [17]:
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [41]:
sum(np.array(weights) == 0) / len(weights)

0.0

In [42]:
len(weights)

10000

In [43]:
mean_array = sum_array / n

In [44]:
for i in range(len(cols)):
    if cols[i] == 'Weight':
        weight_i = i

In [45]:
X_array[0]['Weight']

0     61.0
1     61.0
2     61.0
3     61.0
4     61.0
5     61.0
6     61.0
7     61.0
8     61.0
9     61.0
10    61.0
11    61.0
12    61.0
13    61.0
14    61.0
15    61.0
16    61.0
17    61.0
18    61.0
19    61.0
20    61.0
21    61.0
22    61.0
23    61.0
24    61.0
25    61.0
26    61.0
27    61.0
28    61.0
29    61.0
30    61.0
31    61.0
32    61.0
33    61.0
34    61.0
35    61.0
36    61.0
37    61.0
38    61.0
39    61.0
40    61.0
41    61.0
42    61.0
43    61.0
44    61.0
45    61.0
46    61.0
47    61.0
48    61.0
49    61.0
50    61.0
51    61.0
52    61.0
53    61.0
54    61.0
55    61.0
Name: Weight, dtype: float64

In [46]:
cols

Index(['Age', 'is_F', 'is_urgent', 'is_emergency', 'is_sicu', 'is_tsicu',
       'is_micu', 'is_csru', 'dbp', 'dbp_ind', 'fio2', 'fio2_ind', 'GCS_eye',
       'GCS_eye_ind', 'GCS_motor', 'GCS_motor_ind', 'GCS_verbal',
       'GCS_verbal_ind', 'hr', 'hr_ind', 'map', 'map_ind', 'sbp', 'sbp_ind',
       'spontaneousrr', 'spontaneousrr_ind', 'spo2', 'spo2_ind', 'temp',
       'temp_ind', 'urine', 'urine_ind', 'weight', 'Weight', 'bun', 'bun_ind',
       'magnesium', 'magnesium_ind', 'platelets', 'platelets_ind', 'sodium',
       'sodium_ind', 'alt', 'alt_ind', 'hct', 'hct_ind', 'po2', 'po2_ind',
       'ast', 'ast_ind', 'potassium', 'potassium_ind', 'wbc', 'wbc_ind',
       'bicarbonate', 'bicarbonate_ind', 'creatinine', 'creatinine_ind',
       'lactate', 'lactate_ind', 'pco2', 'pco2_ind', 'bilirubin_total',
       'bilirubin_total_ind', 'glucose', 'glucose_ind', 'inr', 'inr_ind',
       'hgb', 'hgb_ind', 'GCS', 'GCS_ind'],
      dtype='object')

In [47]:
mean_array

array([3.25337824e+03, 2.28487000e+01, 5.16000000e-01, 4.64938000e+01,
       1.17154000e+01, 7.29690000e+00, 2.28859000e+01, 3.02520000e+00,
       3.26002960e+03, 4.58053000e+01, 1.86282040e+03, 6.38150000e+00,
       1.77078900e+02, 1.62788000e+01, 2.84364700e+02, 1.62299000e+01,
       1.87079900e+02, 1.62391000e+01, 4.35913568e+03, 4.76353000e+01,
       4.08939590e+03, 4.60100000e+01, 6.30488910e+03, 4.58165000e+01,
       9.87761850e+02, 4.72887000e+01, 4.95014020e+03, 4.67624000e+01,
       1.88587450e+03, 1.42759000e+01, 7.65209935e+03, 2.91221000e+01,
       4.19724880e+03, 4.18018452e+03, 1.37805360e+03, 4.95920000e+00,
       1.04562820e+02, 4.25070000e+00, 1.17305040e+04, 4.62930000e+00,
       7.10710100e+03, 5.54180000e+00, 3.24184230e+03, 1.52530000e+00,
       1.64234550e+03, 5.61630000e+00, 5.97687150e+03, 2.92180000e+00,
       4.14666650e+03, 1.52530000e+00, 2.07941429e+02, 5.85510000e+00,
       5.57598679e+02, 4.53730000e+00, 1.31816002e+03, 6.86370000e+00,
      

In [48]:
cols

Index(['Age', 'is_F', 'is_urgent', 'is_emergency', 'is_sicu', 'is_tsicu',
       'is_micu', 'is_csru', 'dbp', 'dbp_ind', 'fio2', 'fio2_ind', 'GCS_eye',
       'GCS_eye_ind', 'GCS_motor', 'GCS_motor_ind', 'GCS_verbal',
       'GCS_verbal_ind', 'hr', 'hr_ind', 'map', 'map_ind', 'sbp', 'sbp_ind',
       'spontaneousrr', 'spontaneousrr_ind', 'spo2', 'spo2_ind', 'temp',
       'temp_ind', 'urine', 'urine_ind', 'weight', 'Weight', 'bun', 'bun_ind',
       'magnesium', 'magnesium_ind', 'platelets', 'platelets_ind', 'sodium',
       'sodium_ind', 'alt', 'alt_ind', 'hct', 'hct_ind', 'po2', 'po2_ind',
       'ast', 'ast_ind', 'potassium', 'potassium_ind', 'wbc', 'wbc_ind',
       'bicarbonate', 'bicarbonate_ind', 'creatinine', 'creatinine_ind',
       'lactate', 'lactate_ind', 'pco2', 'pco2_ind', 'bilirubin_total',
       'bilirubin_total_ind', 'glucose', 'glucose_ind', 'inr', 'inr_ind',
       'hgb', 'hgb_ind', 'GCS', 'GCS_ind'],
      dtype='object')

In [52]:
pickle.dump( mean_array, open( "data/feature_means_vasopressor_LOS_6_600.p", "wb" ) )

In [49]:
# change mean to 0 for columns which are indicators.
# find which ones are indicators.
col_indicators = []

for col in X_array[0].columns:
    if '_ind' in col:
        col_indicators.append(1)
    else:
        col_indicators.append(0)

In [50]:
for i in range(len(col_indicators)):
    if col_indicators[i] == 1:
        mean_array[i] = 0

In [51]:
mean_array

array([3.25337824e+03, 2.28487000e+01, 5.16000000e-01, 4.64938000e+01,
       1.17154000e+01, 7.29690000e+00, 2.28859000e+01, 3.02520000e+00,
       3.26002960e+03, 0.00000000e+00, 1.86282040e+03, 0.00000000e+00,
       1.77078900e+02, 0.00000000e+00, 2.84364700e+02, 0.00000000e+00,
       1.87079900e+02, 0.00000000e+00, 4.35913568e+03, 0.00000000e+00,
       4.08939590e+03, 0.00000000e+00, 6.30488910e+03, 0.00000000e+00,
       9.87761850e+02, 0.00000000e+00, 4.95014020e+03, 0.00000000e+00,
       1.88587450e+03, 0.00000000e+00, 7.65209935e+03, 0.00000000e+00,
       4.19724880e+03, 4.18018452e+03, 1.37805360e+03, 0.00000000e+00,
       1.04562820e+02, 0.00000000e+00, 1.17305040e+04, 0.00000000e+00,
       7.10710100e+03, 0.00000000e+00, 3.24184230e+03, 0.00000000e+00,
       1.64234550e+03, 0.00000000e+00, 5.97687150e+03, 0.00000000e+00,
       4.14666650e+03, 0.00000000e+00, 2.07941429e+02, 0.00000000e+00,
       5.57598679e+02, 0.00000000e+00, 1.31816002e+03, 0.00000000e+00,
      

In [29]:
# mean zero, variance zero
# only do this for columns not in ind_cols.

var_array = np.zeros(num_cols)
new_X_arr = []

for patient in X_array:
    # subtract the mean array from each row
    patient = patient.apply(lambda row: row - mean_array, axis=1)
    
    # (x - mu)^2
    for index, row in patient.iterrows():
        var_array += (row.values)**2
        
    new_X_arr.append(patient)
    
X_array = new_X_arr.copy()

In [30]:
X_array[0]

,Age,is_F,is_urgent,is_emergency,is_sicu,is_tsicu,is_micu,is_csru,dbp,dbp_ind,...,bilirubin_total,bilirubin_total_ind,glucose,glucose_ind,inr,inr_ind,hgb,hgb_ind,GCS,GCS_ind
0,-3192.328336,-21.8487,-0.516,-45.4938,-11.7154,-7.2969,-21.8859,-3.0252,-3195.0296,1.0,...,-65.18908,1.0,-7094.6643,1.0,-67.941859,1.0,-542.213349,1.0,-633.5235,0.0
1,-3192.328336,-21.8487,-0.516,-45.4938,-11.7154,-7.2969,-21.8859,-3.0252,-3195.0296,1.0,...,-65.18908,0.0,-7094.6643,0.0,-67.941859,0.0,-542.213349,0.0,-633.5235,1.0
2,-3192.328336,-21.8487,-0.516,-45.4938,-11.7154,-7.2969,-21.8859,-3.0252,-3192.0296,1.0,...,-65.18908,0.0,-7094.6643,0.0,-67.941859,0.0,-542.213349,0.0,-633.5235,0.0
3,-3192.328336,-21.8487,-0.516,-45.4938,-11.7154,-7.2969,-21.8859,-3.0252,-3192.0296,1.0,...,-65.18908,0.0,-7057.6643,1.0,-67.941859,0.0,-542.213349,0.0,-633.5235,0.0
4,-3192.328336,-21.8487,-0.516,-45.4938,-11.7154,-7.2969,-21.8859,-3.0252,-3199.0296,1.0,...,-65.18908,0.0,-7057.6643,0.0,-67.941859,0.0,-542.213349,0.0,-633.5235,0.0
5,-3192.328336,-21.8487,-0.516,-45.4938,-11.7154,-7.2969,-21.8859,-3.0252,-3202.0296,1.0,...,-65.18908,0.0,-7057.6643,0.0,-67.941859,0.0,-542.213349,0.0,-633.5235,1.0
6,-3192.328336,-21.8487,-0.516,-45.4938,-11.7154,-7.2969,-21.8859,-3.0252,-3198.0296,1.0,...,-65.18908,0.0,-7057.6643,0.0,-67.941859,0.0,-542.213349,0.0,-633.5235,0.0
7,-3192.328336,-21.8487,-0.516,-45.4938,-11.7154,-7.2969,-21.8859,-3.0252,-3211.0296,1.0,...,-65.18908,0.0,-7057.6643,0.0,-67.941859,0.0,-542.213349,0.0,-633.5235,0.0
8,-3192.328336,-21.8487,-0.516,-45.4938,-11.7154,-7.2969,-21.8859,-3.0252,-3210.0296,1.0,...,-65.18908,0.0,-7057.6643,0.0,-67.941859,0.0,-542.213349,0.0,-633.5235,0.0
9,-3192.328336,-21.8487,-0.516,-45.4938,-11.7154,-7.2969,-21.8859,-3.0252,-3205.0296,1.0,...,-65.18908,0.0,-7057.6643,0.0,-67.941859,0.0,-542.213349,0.0,-633.5235,0.0


In [31]:
var_array = var_array / n

In [32]:
for i in range(len(col_indicators)):
    if col_indicators[i] == 1:
        var_array[i] = 1
        
var_array

array([5.20538775e+08, 2.56866872e+04, 1.36047456e+01, 1.06311118e+05,
       6.75874888e+03, 2.62472892e+03, 2.57703591e+04, 4.52915683e+02,
       5.22665356e+08, 1.00000000e+00, 1.70673019e+08, 1.00000000e+00,
       1.54211760e+06, 1.00000000e+00, 3.97675896e+06, 1.00000000e+00,
       1.72134007e+06, 1.00000000e+00, 9.34500328e+08, 1.00000000e+00,
       8.22425041e+08, 1.00000000e+00, 1.95493215e+09, 1.00000000e+00,
       4.79832443e+07, 1.00000000e+00, 1.20505375e+09, 1.00000000e+00,
       1.74902865e+08, 1.00000000e+00, 2.88089190e+09, 1.00000000e+00,
       8.66391669e+08, 8.59364529e+08, 9.34158333e+07, 1.00000000e+00,
       5.37687951e+05, 1.00000000e+00, 6.76778531e+09, 1.00000000e+00,
       2.48402783e+09, 1.00000000e+00, 5.19291891e+08, 1.00000000e+00,
       1.32649629e+08, 1.00000000e+00, 1.75702905e+09, 1.00000000e+00,
       8.50969426e+08, 1.00000000e+00, 2.12645736e+06, 1.00000000e+00,
       1.52919871e+07, 1.00000000e+00, 8.54503831e+07, 1.00000000e+00,
      

In [33]:
# Save var array
pickle.dump( var_array, open( "data/feature_vars_vasopressor_LOS_6_600.p", "wb" ) )

In [34]:
# divide by the variance
new_X_arr = []

for patient in X_array:

    patient = patient.apply(lambda row: row / var_array, axis=1)
    new_X_arr.append(patient)
    
X_array = new_X_arr.copy()

In [35]:
X_array[0]

,Age,is_F,is_urgent,is_emergency,is_sicu,is_tsicu,is_micu,is_csru,dbp,dbp_ind,...,bilirubin_total,bilirubin_total_ind,glucose,glucose_ind,inr,inr_ind,hgb,hgb_ind,GCS,GCS_ind
0,-0.000006,-0.000851,-0.037928,-0.000428,-0.001733,-0.00278,-0.000849,-0.006679,-0.000006,1.0,...,-0.000309,1.0,-0.000003,1.0,-0.000278,1.0,-0.000036,1.0,-0.000031,0.0
1,-0.000006,-0.000851,-0.037928,-0.000428,-0.001733,-0.00278,-0.000849,-0.006679,-0.000006,1.0,...,-0.000309,0.0,-0.000003,0.0,-0.000278,0.0,-0.000036,0.0,-0.000031,1.0
2,-0.000006,-0.000851,-0.037928,-0.000428,-0.001733,-0.00278,-0.000849,-0.006679,-0.000006,1.0,...,-0.000309,0.0,-0.000003,0.0,-0.000278,0.0,-0.000036,0.0,-0.000031,0.0
3,-0.000006,-0.000851,-0.037928,-0.000428,-0.001733,-0.00278,-0.000849,-0.006679,-0.000006,1.0,...,-0.000309,0.0,-0.000003,1.0,-0.000278,0.0,-0.000036,0.0,-0.000031,0.0
4,-0.000006,-0.000851,-0.037928,-0.000428,-0.001733,-0.00278,-0.000849,-0.006679,-0.000006,1.0,...,-0.000309,0.0,-0.000003,0.0,-0.000278,0.0,-0.000036,0.0,-0.000031,0.0
5,-0.000006,-0.000851,-0.037928,-0.000428,-0.001733,-0.00278,-0.000849,-0.006679,-0.000006,1.0,...,-0.000309,0.0,-0.000003,0.0,-0.000278,0.0,-0.000036,0.0,-0.000031,1.0
6,-0.000006,-0.000851,-0.037928,-0.000428,-0.001733,-0.00278,-0.000849,-0.006679,-0.000006,1.0,...,-0.000309,0.0,-0.000003,0.0,-0.000278,0.0,-0.000036,0.0,-0.000031,0.0
7,-0.000006,-0.000851,-0.037928,-0.000428,-0.001733,-0.00278,-0.000849,-0.006679,-0.000006,1.0,...,-0.000309,0.0,-0.000003,0.0,-0.000278,0.0,-0.000036,0.0,-0.000031,0.0
8,-0.000006,-0.000851,-0.037928,-0.000428,-0.001733,-0.00278,-0.000849,-0.006679,-0.000006,1.0,...,-0.000309,0.0,-0.000003,0.0,-0.000278,0.0,-0.000036,0.0,-0.000031,0.0
9,-0.000006,-0.000851,-0.037928,-0.000428,-0.001733,-0.00278,-0.000849,-0.006679,-0.000006,1.0,...,-0.000309,0.0,-0.000003,0.0,-0.000278,0.0,-0.000036,0.0,-0.000031,0.0


In [36]:
len(X_array)

15552

In [37]:
# pickle files

pickle.dump(X_array, open( "data/X_vasopressor_LOS_6_600.p", "wb" ) )
pickle.dump(mask_array, open("data/mask_vasopressor_LOS_6_600.p", "wb"))
pickle.dump(y_array, open( "data/y_vasopressor_LOS_6_600.p", "wb" ) )

# Create dictionaries mapping from features names to their means and variances

In [38]:
cols

Index(['Age', 'is_F', 'is_urgent', 'is_emergency', 'is_sicu', 'is_tsicu',
       'is_micu', 'is_csru', 'dbp', 'dbp_ind', 'fio2', 'fio2_ind', 'GCS_eye',
       'GCS_eye_ind', 'GCS_motor', 'GCS_motor_ind', 'GCS_verbal',
       'GCS_verbal_ind', 'hr', 'hr_ind', 'map', 'map_ind', 'sbp', 'sbp_ind',
       'spontaneousrr', 'spontaneousrr_ind', 'spo2', 'spo2_ind', 'temp',
       'temp_ind', 'urine', 'urine_ind', 'weight', 'Weight', 'bun', 'bun_ind',
       'magnesium', 'magnesium_ind', 'platelets', 'platelets_ind', 'sodium',
       'sodium_ind', 'alt', 'alt_ind', 'hct', 'hct_ind', 'po2', 'po2_ind',
       'ast', 'ast_ind', 'potassium', 'potassium_ind', 'wbc', 'wbc_ind',
       'bicarbonate', 'bicarbonate_ind', 'creatinine', 'creatinine_ind',
       'lactate', 'lactate_ind', 'pco2', 'pco2_ind', 'bilirubin_total',
       'bilirubin_total_ind', 'glucose', 'glucose_ind', 'inr', 'inr_ind',
       'hgb', 'hgb_ind', 'GCS', 'GCS_ind'],
      dtype='object')

In [53]:
mu_arr = pickle.load( open( "data/feature_means_vasopressor_LOS_6_600.p", "rb" ) )
var_arr = pickle.load( open( "data/feature_vars_vasopressor_LOS_6_600.p", "rb" ) ) 

In [54]:
# Map from feature name to mu and var.

mu_dict = {}
var_dict = {}

In [41]:
mu_arr.shape

(72,)

In [42]:
len(cols)

72

In [55]:
for i in range(len(cols)):
    mu_dict[cols[i]] = mu_arr[i]
    var_dict[cols[i]] = var_arr[i]

In [56]:
pickle.dump( mu_dict, open( "data/mu_dict_vasopressor_LOS_6_600.p", "wb" ) )

In [45]:
pickle.dump( var_dict, open( "data/var_dict_vasopressor_LOS_6_600.p", "wb" ) )

In [34]:
mu_arr

array([3.25337824e+03, 2.28487000e+01, 5.16000000e-01, 4.64938000e+01,
       1.17154000e+01, 7.29690000e+00, 2.28859000e+01, 3.02520000e+00,
       3.26002960e+03, 4.58053000e+01, 1.86282040e+03, 6.38150000e+00,
       1.77078900e+02, 1.62788000e+01, 2.84364700e+02, 1.62299000e+01,
       1.87079900e+02, 1.62391000e+01, 4.35913568e+03, 4.76353000e+01,
       4.08939590e+03, 4.60100000e+01, 6.30488910e+03, 4.58165000e+01,
       9.87761850e+02, 4.72887000e+01, 4.95014020e+03, 4.67624000e+01,
       1.88587450e+03, 1.42759000e+01, 7.65209935e+03, 2.91221000e+01,
       4.19724880e+03, 4.18018452e+03, 1.37805360e+03, 4.95920000e+00,
       1.04562820e+02, 4.25070000e+00, 1.17305040e+04, 4.62930000e+00,
       7.10710100e+03, 5.54180000e+00, 3.24184230e+03, 1.52530000e+00,
       1.64234550e+03, 5.61630000e+00, 5.97687150e+03, 2.92180000e+00,
       4.14666650e+03, 1.52530000e+00, 2.07941429e+02, 5.85510000e+00,
       5.57598679e+02, 4.53730000e+00, 1.31816002e+03, 6.86370000e+00,
      